In [13]:
import itertools
import pandas as pd
import numpy as np


SIGMA = 0.9


def calculate_reward(row):
    r = 0.
    for i, v in enumerate(row['reward'].split('|')):
        r += np.power(SIGMA, i) * (0 if v == "show" else 1)
    return r


def process_data(data_path, recall_path):
    data = pd.read_csv(data_path, sep='\t', dtype = str)
    for org in ["state", "action", "n_state"]:
        target = org + "_float"
        data[target] = data.apply(
            lambda row: [item for sublist in
                         list(map(lambda t: list(np.array(t.split(','), dtype=np.float64)), row[org].split('|')))
                         for item in sublist
                         ], axis=1
        )
    data['reward_float'] = data.apply(calculate_reward, axis=1)

    recall_data = pd.read_csv(recall_path, sep='\t', dtype = str)
    recall_data['embed_float'] = recall_data.apply(
        lambda row: np.array(row['embedding'][1:-1].split(','), dtype=np.float64).tolist(), axis=1
    )
    recall_tmp = list()
    for idx, row in recall_data.iterrows():
        for i in range(4):
            recall_tmp.append(row['embed_float'][i * 30: (i + 1) * 30])
    recall_tmp.sort()
    recall = list(l for l, _ in itertools.groupby(recall_tmp))

    return data, recall


def gen_samples(id_num=100, sample_size=256):
    from pandas import DataFrame
    ids = np.random.randint(0, 100, size=id_num)
    ids = [str(idx) for idx in ids]
    embeddings = np.random.randn(id_num, 30)
    id_emb_dic = dict(zip(ids, embeddings))
    colunms_name = ['state', 'action', 'reward', 'n_state', 'recall']
    sample_data = []
    for i in range(sample_size):
        state_len = np.random.randint(1, 12)
        state = [str(val) for val in np.random.choice(ids, size=state_len)]
        n_state = [str(val) for val in np.random.choice(ids, size=state_len)]
        action = str(np.random.choice(ids, size=2)[0])
        reward = np.random.rand()
        recall = [action]
        sample_data.append((state, action, reward, n_state, recall))
    data = DataFrame(sample_data, columns=colunms_name)
    write_file(id_emb_dic, sample_data)
    return id_emb_dic, data


def write_file(embedding_file, sample_data):
    with open("embed.csv", "w") as fout:
        head = 'item_id\tembedding\n'
        fout.write(head)
        for item_id, emb in embedding_file.items():
            emb_str = ','.join([str(v) for v in emb])
            outline = '%s\t%s\n' % (item_id, emb_str)
            fout.write(outline)
    print("wrote embedding done")

    with open("train.csv", "w") as fout_sample:
        columns_name = ['state', 'action', 'reward', 'n_state', 'recall']
        head = '%s\n' % ('\t'.join(columns_name))
        fout_sample.write(head)
        for sample in sample_data:
            s_state = '|'.join(sample[0])
            action = sample[1]
            s_reward = str(sample[2])
            s_n_state = '|'.join(sample[3])
            s_recall = '|'.join(sample[4])
            outline = '{}\t{}\t{}\t{}\t{}\n'.format(s_state, action, s_reward, s_n_state, s_recall)
            fout_sample.write(outline)
    print("wrote sample data done")


In [14]:
gen_samples(id_num=100, sample_size=256)

wrote embedding done
wrote sample data done


({'96': array([ 0.11262193, -0.00864939, -0.12147892, -1.6535287 ,  0.94784426,
         -0.17958858,  1.41734215, -1.15442317, -0.46455127, -0.36484953,
          1.21363851, -0.18547857,  1.86091784,  0.31596535, -0.02317252,
          1.08863444, -0.6385979 , -0.76046298,  0.53322343,  0.28457935,
          0.17009799,  0.14313099, -0.68307135,  1.76771031, -1.25643343,
          0.33471705, -0.70548687,  2.12047245, -0.0615184 ,  1.6015234 ]),
  '69': array([-0.89534621,  1.9640951 , -0.73276563,  1.51208265, -1.45210087,
          1.0985829 ,  0.58208758,  0.43998634,  0.21121456,  0.20254101,
         -0.51948586,  0.222736  ,  0.63137116, -0.10620695, -1.77166156,
          0.32081941,  1.17552054, -0.08224675,  1.53463873,  0.09349411,
          0.05296015,  0.24645784,  1.67308392, -0.71655391, -1.23333094,
          0.1943192 ,  0.08380624, -1.3454966 , -1.2020609 ,  0.28246553]),
  '81': array([ 1.52729699,  1.42510976, -0.45750501,  1.2192974 ,  0.98289685,
         -0.6225

In [15]:
recall = pd.read_csv("embed.csv", sep='\t')
recall.head()

,item_id,embedding
0,96,"0.11262193400195745,-0.008649386773077358,-0.1..."
1,69,"-0.895346207612645,1.9640950964352917,-0.73276..."
2,81,"1.5272969902283864,1.4251097557174823,-0.45750..."
3,71,"-0.0050879320467586515,1.5864492890007986,-1.8..."
4,68,"0.11410714716628075,0.3987600956443246,-0.4722..."


In [4]:
recall['embed_float'] = recall.apply(
    lambda row: np.array(row['embedding'][1:-1].split(','), dtype=np.float64).tolist(), axis=1
)
recall

,item_id,embedding,embed_float
0,6,"-0.6627286618645197,1.1359124924916422,-0.1715...","[0.6627286618645197, 1.1359124924916422, -0.17..."
1,29,"-1.1304708990519632,-1.1523589352636994,-0.250...","[1.1304708990519632, -1.1523589352636994, -0.2..."
2,98,"0.2232533044981172,-1.2079186854428503,1.17844...","[0.2232533044981172, -1.2079186854428503, 1.17..."
3,88,"1.49593710584407,-0.43686590139677667,-0.28052...","[0.49593710584407, -0.43686590139677667, -0.28..."
4,27,"1.0989771766012657,0.22801442560616408,0.18250...","[0.0989771766012657, 0.22801442560616408, 0.18..."
5,90,"-0.9981011191624934,-2.3619626818060846,-0.050...","[0.9981011191624934, -2.3619626818060846, -0.0..."
6,37,"0.8902873356336922,0.21457791460339337,-0.4830...","[0.8902873356336922, 0.21457791460339337, -0.4..."
7,19,"-1.718405570735935,-1.4667881874615496,-0.1546...","[1.718405570735935, -1.4667881874615496, -0.15..."
8,11,"0.0541730082368768,-0.5145174110782865,-1.8438...","[0.0541730082368768, -0.5145174110782865, -1.8..."
9,9,"-0.29821654675106124,-0.04760718105889795,0.12...","[0.29821654675106124, -0.04760718105889795, 0...."


In [25]:
recall_tmp = list()
for idx, row in recall.iterrows():
    for i in range(4):
        recall_tmp.append(row['embed_float'][i * 30: (i + 1) * 30])
        

In [31]:
len(recall.iloc[34,2])

30

In [ ]:
data, recall_data = process_data("train.csv", "embed.csv")
data

In [17]:
uu = np.array(data['state_float'].sample(1).values[0])
uu

array([51., 35., 18., 85., 64., 43., 27.,  1., 27.])

In [18]:
uu.reshape((1, 12, 30))

ValueError: cannot reshape array of size 9 into shape (1,12,30)

In [58]:
# for reward, group in data.groupby(['reward']):
#     print(reward)
#     print(group)
    
group.shape

(1, 9)